In [5]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
YOUTUBE_VIDEO = "https://www.youtube.com/watch?v=cdiD-9MMpb0"

In [6]:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")

In [7]:
model.invoke("What is the meaning of life?")

AIMessage(content='The meaning of life is a complex and subjective question that has been pondered by philosophers, theologians, and individuals throughout history. Different people and cultures have different beliefs about the purpose and meaning of life. Some believe that life has inherent meaning or purpose, while others believe that individuals must create their own meaning through their actions and relationships. Ultimately, the meaning of life is a deeply personal and individual question that each person must explore and determine for themselves.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 90, 'prompt_tokens': 14, 'total_tokens': 104, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2961d0b4-e996-4377-9e05-834fd36b5873-0

In [10]:
model.invoke("How much is 2 + 2?")

AIMessage(content='2 + 2 equals 4.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 16, 'total_tokens': 24, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-61785f07-1bae-478d-80e9-fd4e9886a8d5-0', usage_metadata={'input_tokens': 16, 'output_tokens': 8, 'total_tokens': 24, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [11]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke("What is the meaning of life?")


'The meaning of life is a philosophical question that has been debated for centuries. Different cultures, religions, and individuals may have different beliefs about the purpose and meaning of life. Some people believe that the meaning of life is to seek happiness, fulfillment, or spiritual enlightenment, while others believe that life has no inherent meaning and that it is up to each individual to create their own meaning. Ultimately, the meaning of life is a deeply personal and subjective question that each person must answer for themselves.'

In [12]:
from langchain.prompts import ChatPromptTemplate

template = """
Answer the question based on the contect below. If you can't answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""    

prompt = ChatPromptTemplate.from_template(template)
prompt.format(context="Mary's favorite food is pizza.", question="What is Mary's favorite food?")

'Human: \nAnswer the question based on the contect below. If you can\'t answer the question, reply "I don\'t know".\n\nContext: Mary\'s favorite food is pizza.\n\nQuestion: What is Mary\'s favorite food?\n'

In [15]:
chain = prompt | model | parser
chain.invoke({
    "context":"Mary's favorite food is pizza.",
    "question":"What is Mary's favorite food?"
    })

"Mary's favorite food is pizza."

In [16]:
translation_prompt = ChatPromptTemplate.from_template(
    "Translate {answer} to {language}"
)

In [36]:
from operator import itemgetter

# First, answer the question in the specified language
initial_translation_chain = (
    {"answer": chain, "language": itemgetter("language")} | translation_prompt | model | parser
)

initial_answer = initial_translation_chain.invoke({
    "context": "Mary's sister is Susana. She does not have any more siblings.",
    "question": "How many siblings does Mary have?",
    "language": "Polish"
})

# Now, translate the already translated answer to another language
sec_translation_prompt = ChatPromptTemplate.from_template(
    "Translate {answer} to {language} and add info from what language are you translating from"
)

sec_translation_chain = (
    {"answer": itemgetter("answer"), "language": itemgetter("language")} | sec_translation_prompt | model | parser
)

final_translation = sec_translation_chain.invoke({
    "answer": initial_answer,
    "language": "English"
})

print(final_translation)
print(initial_answer)

Mary has one sibling, her sister Susana. (Translated from Polish)
Mary ma jedno rodzeństwo, jej siostra Susana.
